In [1]:
import pandas as pd
import json

In [78]:

  
# Opening JSON file
f = open('./paper_data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
papers = []
title_dicts = []
for i in data:
#     print(i.keys())
#     print(i['csl']['title'])
#     print(i['s2data']['referenceCount'],i['s2data']['citationCount'],i['s2data']['influentialCitationCount'])
#     print(i['s2data']['references'][0][''])
    if 'csl' in i.keys() and i['csl'] != None:
        #print(i['csl'])
        t = i['csl']['title']
        doi = i['doi']
        mini_dict = {doi:{'title':t, 'tags':[], 'modes':[]}}
        title_dicts.append(mini_dict)
  
# Closing file
f.close()

In [79]:
len(title_dicts)

63

In [80]:
title_dicts[0]

{'10.48550/arXiv.2205.02007': {'title': 'A Computational Inflection for Scientific Discovery',
  'tags': [],
  'modes': []}}

In [96]:
ref_dicts = []
for i in data:
    #print(i.keys())
    #print(i['csl']['title'])
    #print(i['s2data']['referenceCount'],i['s2data']['citationCount'],i['s2data']['influentialCitationCount'])
    #print(i['s2data']['references'][0])
    if 'csl' in i.keys() and i['csl'] != None:
        #print(i['csl'])
        t = i['csl']['title']
        doi = i['doi']
        mini_dict = {doi:{'title':t, 'references':[], 'refCount':0}}
        #print(i)
        if type(i['s2data']) == dict:
            #print(i['s2data'].keys())
            mini_dict[doi]['refCount'] = i['s2data']['referenceCount']

            ref_list = i['s2data']['references']
            r_tuples = []
            for r in ref_list:
                if r['paperId'] == None:
                    r_tup = (0,0,0)
                else:
                    r_tup = (r['paperId'],r['citationCount'],r['influentialCitationCount'])
                r_tuples.append(r_tup)
            
            r_tuples = sorted(r_tuples, key=lambda x: x[1], reverse=True)
            mini_dict[doi]['references']= r_tuples
        #print(mini_dict.keys())
        
    ref_dicts.append(mini_dict)
  
# Closing file
f.close()

In [97]:
ref_dicts[0]

{'10.48550/arXiv.2205.02007': {'title': 'A Computational Inflection for Scientific Discovery',
  'references': [('0fc425a8004830fdd7f207efd4fa7a2331d56d3f', 52421, 1930),
   ('330da625c15427c6e42ccfa3b747fb29e5835bf0', 23471, 3698),
   ('228bafce55e6f1cbe2c1df75b1949a1fb9c93eb3', 14882, 717),
   ('aecf01a12624754951e4523b3bc37de480faed81', 14460, 748),
   ('6b85b63579a916f705a8e10a49bd8d849d91b1fc', 5944, 701),
   ('29826ea86aff3025a9b4baaba47ea3e2772c006d', 5183, 155),
   ('dc32a984b651256a8ec282be52310e6bd33d9815', 4718, 502),
   ('c0373426c8e5579dcff60cc0bd930277822edc7d', 2632, 313),
   ('053a25c6b6b63f95fb4e1577f0d4cf26eacae0a1', 1636, 91),
   ('928a61768b492a3614e4c21453e079419d662891', 1318, 67),
   ('5df85ae89af55c6d82a1a14836ea6bcfbfc2c0ec', 1162, 70),
   ('92249d129fe423d292e7a707eaf55309931219f9', 885, 60),
   ('43f4674bded8f35df66c3a6db9ca388735718872', 831, 83),
   ('bec80155723f80c9ff33c9ed1af8c1933162dc4f', 761, 74),
   ('98e65b3454813a7d8bcf20b5b2539ff2c4112bd8', 580, 2

In [100]:

all_ref_dict = {}
for i in ref_dicts:
    #print(i.keys())
    #print(i)
    t = list(i.keys())[0]
    
    for j in i[t]['references']:
        paperId = j[0]
        citCount = j[1]
        infCitCount = j[2]
        if paperId in all_ref_dict.keys():
            #print('Shared Reference!')
            paper_list = all_ref_dict[paperId]['papers']
            paper_list.append(paperId)
            all_ref_dict[paperId]['papers'] = paper_list
        else:
            all_ref_dict[paperId]= {'citationCount':citCount,
                                        'influentialCitationCount':infCitCount,
                                        'papers':[paperId]}
len(all_ref_dict)

2436

In [102]:
import numpy as np

In [146]:
print(len(ref_dicts))

i = ref_dicts[0]
t = list(i.keys())[0]
top_ten = i[t]['references'][:10]
top_ten = [int(i[1]) for i in top_ten]
top_ten_mat = np.array(top_ten)
top_ten_mat = top_ten_mat.reshape((1,10))

all_titles = [i[t]['title']]
for ix, i in enumerate(ref_dicts[1:]):
    t = list(i.keys())[0]
    title = i[t]['title']
    all_titles.append(title)
    top_ten = i[t]['references'][:10]
    top_ten = [int(i[1]) for i in top_ten]
    if len(top_ten)<1:
        top_ten = [0]*10
    elif len(top_ten)<10:
        extra = [0]*(10-len(top_ten))
        top_ten += extra
    top_ten = np.array(top_ten).reshape((1,10))
    
    top_ten_mat = np.concatenate([top_ten_mat,top_ten], axis=0)

top_ten_mat.shape

65


(65, 10)

In [149]:
top_df = pd.DataFrame({'title':all_titles,'top_ref_0':top_ten_mat[:,0],
                     'top_ref_1':top_ten_mat[:,1],'top_ref_2':top_ten_mat[:,2],
                     'top_ref_3':top_ten_mat[:,3],'top_ref_4':top_ten_mat[:,4],
                     'top_ref_5':top_ten_mat[:,5],'top_ref_6':top_ten_mat[:,6],
                     'top_ref_7':top_ten_mat[:,7],'top_ref_8':top_ten_mat[:,8],
                     'top_ref_9':top_ten_mat[:,9]})

top_df.head()

,title,top_ref_0,top_ref_1,top_ref_2,top_ref_3,top_ref_4,top_ref_5,top_ref_6,top_ref_7,top_ref_8,top_ref_9
0,A Computational Inflection for Scientific Disc...,52421,23471,14882,14460,5944,5183,4718,2632,1636,1318
1,Communicating with Interactive Articles,5483,4359,4316,1883,1837,1678,1675,1654,1636,1555
2,Capture &amp; Analysis of Active Reading Behav...,24482,4295,4116,3828,3413,3200,898,339,335,176
3,Literate Programming,1423,702,561,149,55,36,14,9,4,0
4,Exploration and Explanation in Computational N...,8091,2063,1617,1090,898,887,781,759,648,613


### To Dos


- create bar chart based on tags (incorporate theory/tools distinction)
- (find a more robust way to parse titles)
- collect top infleuntial citations for each paper (from semantic scholar)
- aggregate top papers for each 'group' - as defined by bar chart
- create heatmap based on top paper data
- publish to website

In [5]:
topics = ['authoring', 'augmented_lit_search','augmented_visualizations','collaboration',
          'augmented_formatting','accessibility','analytics']
modes = ['theory','tools']

In [6]:
title_dicts[32]

{'10.18653/v1/N18-3011': {'title': 'Construction of the Literature Graph in Semantic Scholar',
  'tags': [],
  'modes': []}}

In [ ]:
source = top_df

pts = alt.selection(type="single", encodings=['x'])

rect = alt.Chart(top_df).mark_rect().encode(
    alt.X('IMDB_Rating:Q', bin=True),
    alt.Y('Rotten_Tomatoes_Rating:Q', bin=True),
    alt.Color('count()',
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
)

circ = rect.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection')
    )
).transform_filter(
    pts
)

bar = alt.Chart(source).mark_bar().encode(
    x='Major_Genre:N',
    y='count()',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=550,
    height=200
).add_selection(pts)

alt.vconcat(
    rect + circ,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

In [74]:
import altair as alt
from vega_datasets import data

source = data.movies.url

pts = alt.selection(type="single", encodings=['x'])

rect = alt.Chart(data.movies.url).mark_rect().encode(
    alt.X('IMDB_Rating:Q', bin=True),
    alt.Y('Rotten_Tomatoes_Rating:Q', bin=True),
    alt.Color('count()',
        scale=alt.Scale(scheme='greenblue'),
        legend=alt.Legend(title='Total Records')
    )
)

circ = rect.mark_point().encode(
    alt.ColorValue('grey'),
    alt.Size('count()',
        legend=alt.Legend(title='Records in Selection')
    )
).transform_filter(
    pts
)

bar = alt.Chart(source).mark_bar().encode(
    x='Major_Genre:N',
    y='count()',
    color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).properties(
    width=550,
    height=200
).add_selection(pts)

alt.vconcat(
    rect + circ,
    bar
).resolve_legend(
    color="independent",
    size="independent"
)

alt.VConcatChart(...)